In [1]:
import xarray as xr
import rydanalysis as ra
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path
import numpy as np
from tqdm.notebook import tqdm#
from scipy.stats import sem
from scipy.integrate import quad

%matplotlib widget

In [1]:
path = Path.cwd().parent / "raw_data"

live_analysis_path = Path.cwd().parent / "Analysis" 
peaks_file = live_analysis_path / "peaks.h5"

NameError: name 'Path' is not defined

In [ ]:
data, peak_df = ra.update_data(path, csv_path="peak_df_2.csv")

2021-05-18 11:32:05.615 INFO    numexpr.utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-05-18 11:32:05.616 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


  0%|          | 0/23 [00:00<?, ?it/s]

## Analysis

In [4]:
variable = data.ryd_data.variables.columns[0]

summary = ra.summarize_peak_description(peak_df)
counts = summary.counts.to_series()
counts_mean = counts.groupby(variable).mean()
counts_sem = counts.groupby(variable).sem(ddof=1)

In [5]:
fig, ax = plt.subplots()

ax.errorbar(counts_mean.index, counts_mean, counts_sem, linestyle='-', marker='o')
ax.set_xlabel(variable)
ax.set_ylabel("ion counts")

fig.savefig("raw_data.png")
#ax.set_xscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
from lmfit.models import GaussianModel, ConstantModel

gaussian_model = GaussianModel()
gaussian_params = gaussian_model.guess(counts_mean.values, x=counts_mean.index)
gaussian_model = gaussian_model + ConstantModel()
gaussian_params.add("c", value=10)
fitres = gaussian_model.fit(counts_mean, params=gaussian_params, x=counts_mean.index)#, weights=1/counts_sem)

fig, ax = plt.subplots()
ax.errorbar(counts_mean.index, counts_mean, counts_sem, linestyle='', marker='o', label="counts")
x_values = np.linspace(counts_mean.index.min(), counts_mean.index.max(), 101)
ax.plot(x_values, fitres.eval(x=x_values))
times = np.linspace(0, 0.2, 110)
plt.text(0.15, 0.9,'center: %.5g  $\pm$ %.5g std: %.5g  $\pm$ %.5g'%(fitres.params['center'].value, fitres.params['center'].stderr, fitres.params['sigma'].value, fitres.params['sigma'].stderr), transform=ax.transAxes)
ax.set_xlabel(variable)
ax.set_ylabel("ion count")

plt.savefig('48P_%s_Fit_Cosine.png'%variable, dpi=600) # add leading zero to the figure name

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
1/(4 * fitres.params["freq"])

0.05909791258860264

In [8]:
1.25 - fitres.params["freq"]/2

-0.8651339281669812

In [9]:
fitres.params["freq"]
fitres.plot_residuals()


<AxesSubplot:title={'center':'Model(damped_cosine)'}, xlabel='tmw[us]', ylabel='residuals'>